# Cell Membrane Segmentation
Given the fluorescence microscopy image showing cell membranes, we'll use watershed segmentation since it's ideal for separating touching objects with bright boundaries.

In [1]:
import numpy as np
from skimage import io, filters, morphology, feature, segmentation
from scipy import ndimage
import matplotlib.pyplot as plt

Create a sample image for demonstration

In [2]:
# Create a sample 512x512 image with cell-like membrane structures
img = np.zeros((512, 512))
rng = np.random.default_rng(42)
centers = rng.integers(0, 512, (20, 2))

for x, y in centers:
    radius = rng.integers(30, 60)
    rr, cc = morphology.disk((y, x), radius, shape=img.shape)
    img[rr, cc] = 1
    
img = filters.gaussian(img, sigma=2)
img = img / img.max()

plt.imshow(img, cmap='gray')
plt.axis('off')
plt.savefig('original.png', bbox_inches='tight', pad_inches=0)
plt.close()

TypeError: disk() got an unexpected keyword argument 'shape'

Enhance membrane edges using gaussian difference

In [ ]:
edges = filters.gaussian(img, sigma=1) - filters.gaussian(img, sigma=2)
edges = edges / edges.max()  # normalize to 0-1
plt.imshow(edges, cmap='gray')
plt.axis('off')
plt.savefig('edges.png', bbox_inches='tight', pad_inches=0)
plt.close()

Find seeds for watershed using local maxima

In [ ]:
distance = ndimage.distance_transform_edt(edges < 0.2)
local_max = feature.peak_local_max(distance, indices=False, min_distance=20)
markers = ndimage.label(local_max)[0]
plt.imshow(markers, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('markers.png', bbox_inches='tight', pad_inches=0)
plt.close()

Apply watershed segmentation

In [ ]:
labels = segmentation.watershed(-distance, markers, mask=edges < 0.3)

# Visualize results
plt.imshow(labels, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmentation.png', bbox_inches='tight', pad_inches=0)
plt.close()

# Save labels
np.save('cell_labels.npy', labels)